<a href="https://colab.research.google.com/github/emerenan/analysing_police_activity/blob/master/police_activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import pandas as pd

# Método para leitura dos dados
def readfile(path):
    """ Método para ler os dados do arquivo e retornar um dataframe.
    Args: \n
        path: localização do arquivo que será lido. \n

    Returns: \n
        Dataframe contendo os dados para analise.

    Raises: \n
        ParserError: retorna as linhas que não puderam ser lidas.
    """

    try:
        df = pd.read_csv(path, error_bad_lines=False, warn_bad_lines=True, parse_dates=True, infer_datetime_format=True)
        """
        print(df.head())
        print('-----------------------------------------------------------------------')
        print(df.isnull().sum())
        print('-----------------------------------------------------------------------')
        print(df.info())
        print('-----------------------------------------------------------------------')
        """
        return df
    except pd.io.common.ParserError:
        print("Dados não puderam ser lidos")

# lendo o arquivo e salvando na variavel data
data = readfile('/content/police.csv')

dict = {}
for label, entry in data.isnull().sum().items():
    # Salvando a forma do dataframe na variavel t
    t = data.shape
    # Salvando a qtd de linhas na variavel rows
    rows = t[0]

    if entry == rows or entry >= rows/2:
        data.drop(label, axis='columns', inplace=True)
        if label in dict.keys():
            dict[label] += "Deletada"
        else:
            dict[label] = "Deletada"
"""
for key, value in dict.items():
    if value == 0:
        print("Não houve colunas deletadas!")
    else:
        print(key, ": ", value)
"""
print('Apagando colunas null!')
print('-----------------------------------------------------------------------')
# Apagando todas as linhas do dataframe, com base nas linhas NUll da coluna driver_gender
print(data.shape) #antes de apagar
data.dropna(subset=['driver_gender'], inplace=True)
print(data.shape) #depois de apagado
print('Apagando linhas Null com base na coluna driver_gender!')
print('-----------------------------------------------------------------------')

# Analisando os tipos de dados do Dataframe para alterar o dtype
data.is_arrested = data.is_arrested.astype('bool')

# combinando stop_date com stop_time e convertendo para string
dates = data.stop_date.str.cat(data.stop_time, sep=' ')

# Criando a coluna stop_datetime e convertendo os valores para datetime
data['stop_datetime'] = pd.to_datetime(dates)

# Trocando a coluna stop_datetime como index do dataframe
data.set_index('stop_datetime', inplace=True)

print('Tratamento de tipos de dados do Dataframe.')
print('-----------------------------------------------------------------------')



Apagando colunas null!
-----------------------------------------------------------------------
(91741, 13)
(86536, 13)
Apagando linhas Null com base na coluna driver_gender!
-----------------------------------------------------------------------
Tratamento de tipos de dados do Dataframe.
-----------------------------------------------------------------------
